In [1]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
df = pd.read_csv(r"../data/GridPuzzle_processed.csv")



In [3]:
df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations


In [40]:
from itertools import combinations
def initialize_probability_tables(num_categories, categories):
    r = 2
    combinations_ls = list(combinations(np.arange(num_categories), r))
    # print(categories)
    # print(combinations_ls)

    matrices = []

    for index, combination in enumerate(combinations_ls):
        # print(combination)
        c1 , c2 = categories[combination[0]].split(", "), categories[combination[1]].split(", ")
        
        matrix =  np.full((len(c1), len(c2)), 1 / max(len(c1), len(c2)))
        matrices.append(matrix)
        
    return np.array(matrices)

print(initialize_probability_tables(df["number_categories"].iloc[0], df["categories"].iloc[0].split("\n"))[0])

[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]


In [4]:
from src.run_agentic_framework import probability_agent
from tqdm import tqdm
import time
def process_query(df):

    results_from_prob_agent = []
    time_taken = []
    for i, row in tqdm(df.iterrows()):
        ques = row["actual_question"]
        clues = row["Clues"]
        categories = row["categories"]
        user_query = """"""
        user_query += ques + clues + "\n" + categories
        start = time.time()
        res_ = probability_agent(user_query)
        end = time.time()
        diff = end - start
        results_from_prob_agent.append(res_)
        time_taken.append(diff)
    
    if results_from_prob_agent != []:
        df["prob_agent_results"] = pd.Series(results_from_prob_agent)
        df["time_taken"] = pd.Series(time_taken)
        df.to_csv(r"../data/GridPuzzele_processed_with_prob_agent_results.csv", index=False)

    return

process_query(df)

0it [00:00, ?it/s]INFO:google_genai.models:AFC is enabled with max remote calls: 10.


INFO:google_genai.models:AFC remote call 1 is done.
1it [00:18, 18.81s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
2it [00:30, 14.77s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
3it [00:50, 17.11s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
4it [01:06, 16.57s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
5it [01:26, 17.68s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
6it [01:45, 18.17s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
7it [02:23, 24.83s/it]INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_g

In [24]:
df

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,462.json,12782,"Using only the clues below, match the months t...",January | Sanchez | Oscar | Quenray\nFebruary ...,| January | Commander Hensley | Oscar Battalio...,Let's break down the puzzle step by step:\n \n...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 7, th...","Step-by-step solution:\n \n 1. From clue 7, th...","Using only the clues below, match the months t...",Clues:\n1. The battalion led by Commander Ingr...,"months : January, February, March, April, May,...",13,4,months commanders battalions locations
270,462.json,13527,The local aerophobe (one with a fear of flying...,January | Troy | wishbone | Pennsylvania\nFebr...,| January | Maurice | coin | Virginia |\n |---...,Let's break down the puzzle step by step:\n \n...,"Step-by-step solution:\n \n 1. From clue 1, we...","Step-by-step solution:\n \n 1. From clue 1, th...","Step-by-step solution:\n \n 1. From clue 1, th...",The local aerophobe (one with a fear of flying...,Clues:\n1. The flier leaving in Ma

In [7]:
ques = df["actual_question"].iloc[0]
clues = df["Clues"].iloc[0]
categories = df["categories"].iloc[0]
from src.run_agentic_framework import probability_agent, validator_agent


In [8]:
user_query = """"""
user_query += ques + "\n" + categories + "\n" + clues #+ "\n" + "\n" + "Let's think step by step"
agent_reply = probability_agent(user_query)

print(agent_reply)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.


Okay, I understand the puzzle. I need to create probability matrices for Vintages-Wines, Vintages-Types, and Wines-Types, and then use the clues to fill in probabilities, normalize, and solve. Let's begin!

**Initial Probability Matrices:**

**1. Vintages - Wines**

|          | Annata Branco | Bianca Flaux | Ece Suss | Vendemmia |
|----------|---------------|--------------|----------|-----------|
| 1984     | 0.25          | 0.25         | 0.25     | 0.25      |
| 1988     | 0.25          | 0.25         | 0.25     | 0.25      |
| 1992     | 0.25          | 0.25         | 0.25     | 0.25      |
| 1996     | 0.25          | 0.25         | 0.25     | 0.25      |

**2. Vintages - Types**

|          | Gewurztraminer | Merlot | Pinot Noir | Riesling |
|----------|----------------|--------|------------|----------|
| 1984     | 0.25           | 0.25   | 0.25       | 0.25     |
| 1988     | 0.25           | 0.25   | 0.25       | 0.25     |
| 1992     | 0.25           | 0.25   | 0.25       | 0

In [6]:
from pprint import pprint
pprint(user_query)

('Nicholas and Edith are having a small dinner party this evening at their '
 "home in Cedar Valley, and they've decided to open a select number of rare "
 'wines from their personal collection to celebrate the occasion. Using only '
 'the clues below, match the vintages to the options from wines and types. '
 'Remember, as with all grid-based logic puzzles, no option in any category '
 'will ever be used more than once.Clues:\n'
 '1. The Ece Suss was bottled sometime after the Annata Branco.\n'
 '2. The Bianca Flaux was bottled 4 years before the Vendemmia.\n'
 '3. The 1988 bottle is a pinot noir.\n'
 '4. The merlot is either the Annata Branco or the Bianca Flaux.\n'
 '5. The Bianca Flaux was bottled sometime after the Ece Suss.\n'
 '6. The 1984 bottle is a gewurztraminer.\n'
 'vintages : 1984, 1988, 1992, 1996.\n'
 'wines : Annata Branco, Bianca Flaux, Ece Suss, Vendemmia.\n'
 'types : gewurztraminer, merlot, pinot noir, riesling.')


In [16]:
pprint(df["question"].iloc[0])

('Nicholas and Edith are having a small dinner party this evening at their '
 "home in Cedar Valley, and they've decided to open a select number of rare "
 'wines from their personal collection to celebrate the occasion. Using only '
 'the clues below, match the vintages to the options from wines and types. '
 'Remember, as with all grid-based logic puzzles, no option in any category '
 'will ever be used more than once.\n'
 '\n'
 'vintages : 1984, 1988, 1992, 1996.\n'
 'wines : Annata Branco, Bianca Flaux, Ece Suss, Vendemmia.\n'
 'types : gewurztraminer, merlot, pinot noir, riesling.\n'
 '\n'
 'Clues:\n'
 '1. The Ece Suss was bottled sometime after the Annata Branco.\n'
 '2. The Bianca Flaux was bottled 4 years before the Vendemmia.\n'
 '3. The 1988 bottle is a pinot noir.\n'
 '4. The merlot is either the Annata Branco or the Bianca Flaux.\n'
 '5. The Bianca Flaux was bottled sometime after the Ece Suss.\n'
 '6. The 1984 bottle is a gewurztraminer.\n'
 '\n'
 'While answering use the 

In [8]:
res_ = probability_agent(user_query)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.


In [10]:
validation_query = """"""
validation_query += "Clues and Categories: \n" + clues + "\n" + categories + "\n"
validation_query += "Probability Agent Response: \n" + agent_reply

In [11]:
new_res = validator_agent(validation_query)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.


In [12]:
print(new_res)

## Verification of Logical Reasoning and Probability Calculations:

The agent's solution demonstrates a good understanding of the problem and attempts to use probability matrices to solve it. However, there are several errors in the reasoning and application of the clues, leading to incorrect deductions and an inconsistent final answer.

Here's a detailed breakdown of the errors and a corrected solution:

**1. Initial Probability Matrices:**

*   The initial probability matrices are correctly set up with equal probabilities for each cell.

**2. Clue 1: The Ece Suss was bottled sometime after the Annata Branco.**

*   This clue is correctly interpreted as establishing a relative order.

**3. Clue 2: The Bianca Flaux was bottled 4 years before the Vendemmia.**

*   The agent correctly identifies the possible year pairings for Bianca Flaux and Vendemmia.
*   The updates to the Vintages-Wines matrix based on this clue are correct.
*   The normalization of the Vintages-Wines matrix is also 

In [14]:
import numpy as np

def create_probability_tables(vintages, wines, types):
    num_wines = len(wines)
    num_types = len(types)
    num_vintages = len(vintages)
    
    vintage_wine = np.full((num_vintages, num_wines), 1 / num_wines)
    vintage_type = np.full((num_vintages, num_types), 1 / num_types)
    wine_type = np.full((num_wines, num_types), 1 / num_types)
    
    return vintage_wine, vintage_type, wine_type

def normalize(table):
    # Normalize rows to sum to 1
    for i in range(table.shape[0]):
        row_sum = table[i, :].sum()
        if row_sum > 0:
            table[i, :] /= row_sum

    # Normalize columns to sum to 1
    for j in range(table.shape[1]):
        col_sum = table[:, j].sum()
        if col_sum > 0:
            table[:, j] /= col_sum

def apply_constraints(vintage_wine, vintage_type, wine_type, vintages, wines, types):
    wine_vintage_map = {"Annata Branco": 1984, "Ece Suss": 1988, "Bianca Flaux": 1992, "Vendemmia": 1996}
    type_vintage_map = {1984: "gewurztraminer", 1988: "pinot noir"}
    wine_type_map = {"merlot": ["Annata Branco", "Bianca Flaux"]}
    
    for wine, vintage in wine_vintage_map.items():
        vintage_idx = vintages.index(vintage)
        wine_idx = wines.index(wine)
        vintage_wine[vintage_idx, wine_idx] = 1
    
    normalize(vintage_wine)
    
    for vintage, wtype in type_vintage_map.items():
        vintage_idx = vintages.index(vintage)
        type_idx = types.index(wtype)
        vintage_type[vintage_idx, type_idx] = 1
    
    normalize(vintage_type)
    
    for wtype, wine_list in wine_type_map.items():
        type_idx = types.index(wtype)
        for wine in wine_list:
            wine_idx = wines.index(wine)
            wine_type[wine_idx, type_idx] = 1
    
    normalize(wine_type)

def main():
    vintages = [1984, 1988, 1992, 1996]
    wines = ["Annata Branco", "Bianca Flaux", "Ece Suss", "Vendemmia"]
    types = ["gewurztraminer", "merlot", "pinot noir", "riesling"]
    
    vintage_wine, vintage_type, wine_type = create_probability_tables(vintages, wines, types)
    apply_constraints(vintage_wine, vintage_type, wine_type, vintages, wines, types)
    
    print("Vintage vs Wine:")
    print(vintage_wine)
    print("\nVintage vs Type:")
    print(vintage_type)
    print("\nWine vs Type:")
    print(wine_type)

main()


Vintage vs Wine:
[[0.57142857 0.14285714 0.14285714 0.14285714]
 [0.14285714 0.14285714 0.57142857 0.14285714]
 [0.14285714 0.57142857 0.14285714 0.14285714]
 [0.14285714 0.14285714 0.14285714 0.57142857]]

Vintage vs Type:
[[0.47058824 0.18181818 0.11764706 0.18181818]
 [0.11764706 0.18181818 0.47058824 0.18181818]
 [0.20588235 0.31818182 0.20588235 0.31818182]
 [0.20588235 0.31818182 0.20588235 0.31818182]]

Wine vs Type:
[[0.18181818 0.34782609 0.18181818 0.18181818]
 [0.18181818 0.34782609 0.18181818 0.18181818]
 [0.31818182 0.15217391 0.31818182 0.31818182]
 [0.31818182 0.15217391 0.31818182 0.31818182]]


In [12]:
df["answer"].iloc[0]

'1984 | Annata Branco | gewurztraminer\n1988 | Ece Suss | pinot noir\n1992 | Bianca Flaux | merlot\n1996 | Vendemmia | riesling'

In [30]:
df = pd.read_csv(r"../data/GridPuzzele_processed_with_prob_agent_results.csv")
pprint(df["prob_agent_results"].iloc[8])

('Okay, I will create probability matrices to represent the relationships '
 'between the categories (Days, Names, Departures) and update them based on '
 'the clues provided.\n'
 '\n'
 '**Initial Matrices (All probabilities set to 0.25, representing equal '
 'likelihood):**\n'
 '\n'
 '**Matrix 1: Days vs. Names**\n'
 '\n'
 '|          | Allen | Eugene | Mitchell | Roderick |\n'
 '|----------|-------|--------|----------|----------|\n'
 '| June 10  | 0.25  | 0.25   | 0.25     | 0.25     |\n'
 '| June 11  | 0.25  | 0.25   | 0.25     | 0.25     |\n'
 '| June 12  | 0.25  | 0.25   | 0.25     | 0.25     |\n'
 '| June 13  | 0.25  | 0.25   | 0.25     | 0.25     |\n'
 '\n'
 '**Matrix 2: Days vs. Departures**\n'
 '\n'
 '|          | Epworth | Farley | Hardwick | Jackman |\n'
 '|----------|---------|--------|----------|---------|\n'
 '| June 10  | 0.25    | 0.25   | 0.25     | 0.25    |\n'
 '| June 11  | 0.25    | 0.25   | 0.25     | 0.25    |\n'
 '| June 12  | 0.25    | 0.25   | 0.25     | 0.25 

In [11]:
pprint(df["Clues"].iloc[0])

('Clues:\n'
 '1. The Ece Suss was bottled sometime after the Annata Branco.\n'
 '2. The Bianca Flaux was bottled 4 years before the Vendemmia.\n'
 '3. The 1988 bottle is a pinot noir.\n'
 '4. The merlot is either the Annata Branco or the Bianca Flaux.\n'
 '5. The Bianca Flaux was bottled sometime after the Ece Suss.\n'
 '6. The 1984 bottle is a gewurztraminer.')


In [31]:
pprint(df["answer"].iloc[8])

('June 10 | Eugene | Farley\n'
 'June 11 | Allen | Jackman\n'
 'June 12 | Mitchell | Hardwick\n'
 'June 13 | Roderick | Epworth')


In [2]:
import pandas as pd
df =pd.read_csv(r"../data/GridPuzzele_processed_with_prob_agent_results.csv")

In [4]:
from pprint import pprint
prob_agent_results = df["prob_agent_results"].values.tolist()

pprint(prob_agent_results[0])

('Okay, I understand the puzzle. I need to create probability matrices for '
 'each pair of categories (vintages, wines, and types), process the clues to '
 'update the probabilities in these matrices, and normalize the rows and '
 'columns after each update.\n'
 '\n'
 'Here are the initial probability matrices:\n'
 '\n'
 '**1. Vintages-Wines Matrix:**\n'
 '\n'
 '|             | Annata Branco | Bianca Flaux | Ece Suss | Vendemmia |\n'
 '|-------------|---------------|--------------|----------|-----------|\n'
 '| 1984        | 0.25          | 0.25         | 0.25     | 0.25      |\n'
 '| 1988        | 0.25          | 0.25         | 0.25     | 0.25      |\n'
 '| 1992        | 0.25          | 0.25         | 0.25     | 0.25      |\n'
 '| 1996        | 0.25          | 0.25         | 0.25     | 0.25      |\n'
 '\n'
 '**2. Vintages-Types Matrix:**\n'
 '\n'
 '|             | gewurztraminer | merlot | pinot noir | riesling |\n'
 '|-------------|----------------|--------|------------|----------|\

In [5]:
pprint(prob_agent_results[15])

('Okay, I will create the probability matrices and process the clues '
 'step-by-step to solve the puzzle.\n'
 '\n'
 '**1. Define the Categories and Create Initial Probability Matrices:**\n'
 '\n'
 '*   **Categories:** Delivery Dates, Customers, Cake Shapes\n'
 '\n'
 '*   **Matrices:**\n'
 '\n'
 '    *   Delivery Dates - Customers\n'
 '    *   Delivery Dates - Cake Shapes\n'
 '    *   Customers - Cake Shapes\n'
 '\n'
 '**2. Initial Matrices (All probabilities start at 0.25 or 1/4):**\n'
 '\n'
 '| Delivery Dates - Customers | Danielson | Estrada | Ingram | Nolan |\n'
 '| :------------------------- | :-------- | :------ | :----- | :---- |\n'
 '| October 5                  | 0.25      | 0.25    | 0.25   | 0.25  |\n'
 '| October 6                  | 0.25      | 0.25    | 0.25   | 0.25  |\n'
 '| October 7                  | 0.25      | 0.25    | 0.25   | 0.25  |\n'
 '| October 8                  | 0.25      | 0.25    | 0.25   | 0.25  |\n'
 '\n'
 '| Delivery Dates - Cake Shapes | Baseball Gl

In [6]:
pprint(prob_agent_results[10])

('Okay, I will create the probability matrices and process the clues '
 'step-by-step to solve the puzzle.\n'
 '\n'
 '**1. Define the Categories and Create Initial Probability Matrices**\n'
 '\n'
 'We have three categories: Months, Owners, and Dog Breeds. This means we need '
 'to create the following matrices:\n'
 '\n'
 '*   Months-Owners\n'
 '*   Months-Dog Breeds\n'
 '*   Owners-Dog Breeds\n'
 '\n'
 'Initially, each cell in the matrices will have a probability of 0.25, '
 'representing equal uncertainty.\n'
 '\n'
 '**Months-Owners Matrix**\n'
 '\n'
 '|          | January | February | March | April |\n'
 '| :------- | :------ | :------- | :---- | :---- |\n'
 '| Kayla    | 0.25    | 0.25     | 0.25  | 0.25  |\n'
 '| Lydia    | 0.25    | 0.25     | 0.25  | 0.25  |\n'
 '| Rosie    | 0.25    | 0.25     | 0.25  | 0.25  |\n'
 '| Violet   | 0.25    | 0.25     | 0.25  | 0.25  |\n'
 '\n'
 '**Months-Dog Breeds Matrix**\n'
 '\n'
 '|             | January | February | March | April |\n'
 '| :---

In [7]:
pprint(prob_agent_results[100])

('Okay, I will create the probability matrices and process the clues '
 'step-by-step to solve the puzzle.\n'
 '\n'
 '**1. Define the Categories and Create Initial Probability Matrices**\n'
 '\n'
 'The categories are: Months, Fliers, and Lucky Charms.  I will create '
 'matrices for each pair of categories:\n'
 '\n'
 '*   Months-Fliers\n'
 '*   Months-Lucky Charms\n'
 '*   Fliers-Lucky Charms\n'
 '\n'
 'Initially, each cell in the matrices will have a probability of 1/5 = 0.2, '
 'representing equal likelihood.\n'
 '\n'
 '**Months-Fliers Matrix**\n'
 '\n'
 '|          | Brandi | Christie | Eloise | Florence | Jaime |\n'
 '| :------- | :----- | :------- | :----- | :------- | :---- |\n'
 '| January  | 0.2    | 0.2      | 0.2    | 0.2      | 0.2   |\n'
 '| February | 0.2    | 0.2      | 0.2    | 0.2      | 0.2   |\n'
 '| March    | 0.2    | 0.2      | 0.2    | 0.2      | 0.2   |\n'
 '| April    | 0.2    | 0.2      | 0.2    | 0.2      | 0.2   |\n'
 '| May      | 0.2    | 0.2      | 0.2    

In [8]:
pprint(prob_agent_results[19])

('Okay, I will create the probability matrices and process the clues '
 'step-by-step to solve the puzzle.\n'
 '\n'
 '**1. Define the Categories and Create Initial Probability Matrices**\n'
 '\n'
 'We have three categories: Years, Astronomers, and Comets. This means we need '
 'to create the following matrices:\n'
 '\n'
 '*   Years-Astronomers\n'
 '*   Years-Comets\n'
 '*   Astronomers-Comets\n'
 '\n'
 'Here are the initial matrices, with all probabilities set to 0.25 (since '
 'each option has an equal chance of being correct at the start):\n'
 '\n'
 '**Years-Astronomers**\n'
 '\n'
 '|          | Hal Gregory | Jack Ingram | Ken Jones | Underwood |\n'
 '| :------- | :---------- | :---------- | :-------- | :--------- |\n'
 '| **2008** | 0.25        | 0.25        | 0.25      | 0.25       |\n'
 '| **2009** | 0.25        | 0.25        | 0.25      | 0.25       |\n'
 '| **2010** | 0.25        | 0.25        | 0.25      | 0.25       |\n'
 '| **2011** | 0.25        | 0.25        | 0.25      | 0